# Конспект курса [Applied text mining in Python](https://www.coursera.org/learn/python-text-mining/)
## Общие впечатления
Лектор - индус с жутким акцентом. Смотреть видео крайне сложно. По американской традиции некторые вещи повторяются по три раза.
Практические задания довольно интересные.
При этом про первое задание есть куча негативных отзывов людей, которые потратили на него по 20-40 часов вместо 3.
Во втором есть задачи по темам, которые вообще никак не освещались в теории. Проще было заменить всё это на фразу "Разберитесь сами в NLTK"

## Синтаксические интересности
Смотрите как изящно меняются значения в столбце с помощью функции **np.where**:

In [9]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


spam_data = pd.read_csv('spam.csv')
spam_data.head(5)

,text,target
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [10]:
spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(5)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


Для вычисления средней длины спам-письма можно использовать numpy и list(map(<функция>,<данные>))

In [18]:
length_spam = list(map(len,spam_data["text"][spam_data["target"]==1]))
np.mean(length_spam)

138.8661311914324

А можно и pandas с последовательностью действий

In [19]:
spam_data["text"][spam_data["target"]==1].str.len().mean()

138.8661311914324

In [11]:


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

## Vectorizer и его фичи
Два способа добаться до ключей словаря и найти максимальный по длине из них:

In [13]:
vect = CountVectorizer()
X = vect.fit_transform(X_train)

vdf = pd.DataFrame.from_dict(vect.vocabulary_, orient="index")
vdf["token"] = vdf.index
vdf["token_len"]=vdf["token"].str.len()
print(vdf["token_len"].max())

print(vdf["token"].str.len().argmax())

#vect = CountVectorizer().fit(X_train)
feature_names = np.array(vect.get_feature_names())
length = list(map(len,feature_names))
print(feature_names[np.argmax(length)]) #Your answer here


32
com1win150ppmx3age16subscription
com1win150ppmx3age16subscription


### написать про get_feature_names

## roc_auc_score и прочие.
Сначала ground truth, потом предсказание

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

clf = MultinomialNB(alpha=0.1)
clf.fit(X, y_train)


print (roc_auc_score(y_test, clf.predict(vect.transform(X_test))) )

0.972081218274


# WordNet